### Pre Processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('prix_voiture.csv')

In [4]:
df.columns

Index(['identifiant', 'etat_de_route', 'carburant', 'turbo', 'nombre_portes',
       'type_vehicule', 'roues_motrices', 'emplacement_moteur', 'empattement',
       'longueur', 'largeur', 'hauteur', 'poids_vehicule', 'type_moteur',
       'nombre_cylindres', 'taille_moteur', 'systeme_carburant',
       'taux_alésage', 'course', 'taux_compression', 'chevaux', 'tour_moteur',
       'consommation_ville', 'consommation_autoroute', 'prix', 'marque',
       'modèle'],
      dtype='object')

In [35]:
df["carburant"].unique()
df["modèle"].nunique()


141

In [5]:
# Define X and y variables
y = df['prix']
X = df.drop(['identifiant', 'prix'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)


# Create a linear regression model and fit it to the data
model = LinearRegression()
model.fit(X, y)

# Print the coefficients of the model
print(model.coef_)

ValueError: could not convert string to float: 'essence'